# 萃取字詞的四大方法
# 首先要先確定有 [字典] 和 [停用詞] 的txt檔

In [1]:
import os
import jieba
import jieba.analyse
import jieba.posseg as pseg  
jieba.set_dictionary('dict.big.tra.txt')  #預設字典
jieba.load_userdict('dict_twstd_tfidf.txt')  #中文分詞詞庫(TFIDF)
jieba.load_userdict('dict_ntusd_pos.txt') #NTUSD_負向
jieba.load_userdict('dict_ntusd_nag.txt') #NTUSD_正向
jieba.load_userdict('dict_twedu_dict.txt') #教育部《重編國語辭典 修訂本》


#jieba.load_userdict('dict_20160111_NotReallyUseful.txt')  
#中文分詞詞庫(TFIDF) + 百度分詞詞庫 + 69萬中文大辭典 + 中文热门词库(155073) 
#不是很好用，可備著使用


jieba.analyse.set_stop_words("stopwords_1227.txt") #停用詞

Building prefix dict from C:\Users\BigData\dict.big.tra.txt ...
DEBUG:jieba:Building prefix dict from C:\Users\BigData\dict.big.tra.txt ...
Loading model from cache c:\users\bigdata\appdata\local\temp\jieba.ud8ec30fabaf60e161f06b3552aab2f0e.cache
DEBUG:jieba:Loading model from cache c:\users\bigdata\appdata\local\temp\jieba.ud8ec30fabaf60e161f06b3552aab2f0e.cache
Loading model cost 0.278 seconds.
DEBUG:jieba:Loading model cost 0.278 seconds.
Prefix dict has been built succesfully.
DEBUG:jieba:Prefix dict has been built succesfully.


# 方法一：計算Jieba TFIDF值

In [2]:
# 單一景點使用[Jieba TFIDF]的方法
def jiebastat(attraction,rankNum):
#轉換象山資料,並切詞,並使用JIEBA內建的tfidf統計排名
    f = open(attraction,'r')  #讀取檔案
    f1 = f.read()  #接住
    f2 = ''.join(f1.split())  #把空白全去掉
    #print f2
    words=jieba.cut(f2)  #斷詞
    n = ' '.join(words)  #把斷詞結果用空白隔開
    n1 = n.encode('utf-8')  #編碼
    f.close()


    temp = []
    temp2 =[]
    x = jieba.analyse.extract_tags(n1, topK=rankNum, withWeight=True)
    for u in x:
        #print u[0],"\t",u[1]
        temp.append(u[0].encode('utf-8'))  #存詞彙
        temp2.append(u[1])  #存權重

    #統計字頻
    dic={}
    for w in n1.split(' '):
        if w not in dic:
            dic[w] = 1
        else:
            dic[w] = dic[w] + 1
    import operator
    words_freq = sorted(dic.iteritems(),key=operator.itemgetter(1),reverse=True)
    print attraction,"\n總詞量：",len(words_freq)
    

    #根據Jieba tf-idf去看字頻排名
    print "=========字頻統計(根據jieba TFIDF)========="
    print "排名","\t","總排","\t","詞彙","\t","出現次數","  ","權重","\t\t\t","比率"
    print "- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -"
    num = 0
    num3 = -1
    for ele in words_freq:
        num += 1
        if ele[0] in temp:
            num3 += 1
            print num3+1,"\t",num,"\t",ele[0],"\t",ele[1],"\t","   ",temp2[num3],\
            "\t",float(int(( float(ele[1]) / float(len(words_freq)) ) *10000))/100,'%'

# 比對多個景點[Jieba TFIDF]的方法
def mutijiebastat(attrlist,RankNum):
    for i in range(0,len(attrlist)):
        jiebastat(attrlist[i],RankNum)
        print '\n'

In [66]:
lis = ['mountain.txt','lungshanTemple.txt','maokongGondola.txt']
mutijiebastat(lis,10)

mountain.txt 
總詞量： 4487
=========字頻統計(根據jieba TFIDF)=========
排名 	總排 	詞彙 	出現次數    權重 			比率
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
1 	13 	101 	242 	    0.266101336985 	5.39 %
2 	23 	象山 	156 	    0.147262902206 	3.47 %
3 	47 	夜景 	87 	    0.0912661610321 	1.93 %
4 	48 	大象 	86 	    0.0796174348974 	1.91 %
5 	50 	檢視 	83 	    0.072573091905 	1.84 %
6 	54 	美景 	74 	    0.0695205415996 	1.64 %
7 	56 	日落 	73 	    0.0651036522607 	1.62 %
8 	60 	風景 	66 	    0.0630026876112 	1.47 %
9 	68 	很棒 	60 	    0.0615771688891 	1.33 %
10 	76 	臺北市 	56 	    0.061413294314 	1.24 %


lungshanTemple.txt 
總詞量： 4970
=========字頻統計(根據jieba TFIDF)=========
排名 	總排 	詞彙 	出現次數    權重 			比率
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
1 	6 	寺廟 	459 	    0.502494348336 	9.23 %
2 	13 	龍山寺 	213 	    0.233183651842 	4.28 %
3 	22 	地方 	141 	    0.114949687528 	2.83 %
4 	30 	臺灣 	105 	    0.086515551715 	2.11 %
5 	34 	夜市 	94 	    0.0634960178725 	1.89 %
6 	35 	值得 	91 	    0.0624012589437 	

# 方法二：計算Scikit-Learn的TFIDF值

In [3]:
def wordsToList(attfile):
    f = open(attfile,'r')
    f1 = f.read()
    f2 = ''.join(f1.split())
    #print f2
    words=jieba.cut(f2)
    n = ' '.join(words)
    n1 = n.encode('utf-8')
    return n1
    f.close()

def stopWordsList(stopwordfile):
    li = []
    fid =  open(stopwordfile,'r')
    for line in fid:
        li.append(line.strip().decode('utf-8'))#要解開utf-8,才能成功給scikitlearn用
    fid.close()
    return li

def CompareTFIDF(listInput,stopWordsFile,rankNum):
    from sklearn import feature_extraction  
    from sklearn.feature_extraction.text import TfidfTransformer  
    from sklearn.feature_extraction.text import CountVectorizer
    
    #創造一個空矩陣
    corpus = []
    
    #把傳入的List依序讀出，使用方法把這些東西全都讀出來
    for each in range(0,len(listInput)):
        corpus.append(wordsToList(listInput[each]))

    vectorizer=CountVectorizer(analyzer='word',stop_words=stopWordsList(stopWordsFile))  #該類會將文本中的詞語轉換為詞頻矩陣，矩陣元素a[i][j] 表示j詞在i類文本下的詞頻  
    x = vectorizer.fit_transform(corpus)  #將文本轉為詞頻矩陣 
    transformer=TfidfTransformer()  #該類會統計每個詞語的tf-idf權值 
    tfidf=transformer.fit_transform(x)  #計算tf-idf
    word=vectorizer.get_feature_names()  #獲取詞袋模型中的所有詞語
    weight=tfidf.toarray()   #將tf-idf矩陣抽取出來，元素a[i][j]表示j詞在i類文本中的tf-idf權重  

    print '-------以下輸出各景點中TFIDF值最高的幾個詞語------\n'
    
    #打印每類文本的tf-idf詞語權重，第一個for遍歷所有文本，第二個for便利某一類文本下的詞語權重
    import operator
    for i in range(len(weight)):
        print listInput[i],"\n------文章內詞語tf-idf權重------" 
        print "排名",'\t',"詞語","\t","權重"
        print "- - - - - - - - - - - - - - - -"
        dic = {}
        for j in range(len(word)):  
            dic[word[j]] = weight[i][j]
        words_freq = sorted(dic.iteritems(),key=operator.itemgetter(1),reverse=True)
        num2 = 0
        for ele in words_freq[0:rankNum]:
            num2 += 1
            print num2,"\t",ele[0],"\t",ele[1]
        print '\n'

In [43]:
lis = ['mountain.txt','lungshanTemple.txt','maokongGondola.txt']
CompareTFIDF(lis,'stopwords_1227.txt',10)

-------以下輸出各景點中TFIDF值最高的幾個詞語------

mountain.txt 
------文章內詞語tf-idf權重------
排名 	詞語 	權重
- - - - - - - - - - - - - - - -
1 	101 	0.419783594375
2 	象山 	0.348452306491
3 	地方 	0.234176798515
4 	城市 	0.213361083092
5 	大象 	0.192095502296
6 	值得 	0.168260366341
7 	夜景 	0.150913936821
8 	檢視 	0.143975365013
9 	晚上 	0.131832864349
10 	美景 	0.128363578445


lungshanTemple.txt 
------文章內詞語tf-idf權重------
排名 	詞語 	權重
- - - - - - - - - - - - - - - -
1 	寺廟 	0.622179753309
2 	龍山寺 	0.488852121263
3 	地方 	0.191127113762
4 	臺灣 	0.142328701737
5 	夜市 	0.127418075841
6 	值得 	0.123351541506
7 	文化 	0.0977461960654
8 	祈禱 	0.0925097927047
9 	歷史 	0.0907643249178
10 	捷運 	0.0786196638168


maokongGondola.txt 
------文章內詞語tf-idf權重------
排名 	詞語 	權重
- - - - - - - - - - - - - - - -
1 	纜車 	0.516165487179
2 	貢多拉 	0.351399397095
3 	貓空 	0.33188958428
4 	動物園 	0.287509000335
5 	水晶 	0.260060925648
6 	等待 	0.125423389408
7 	玻璃 	0.119634617589
8 	車廂 	0.117978858952
9 	小屋 	0.110367319665
10 	風景 	0.103396447525




# 方法三 計算字頻 - 文章斷詞的原始字頻

In [4]:
def jiebafrestat(attraction,rankNum):
#轉換象山資料,並切詞,並使用JIEBA內建的tfidf統計排名
    f = open(attraction,'r')  #讀取檔案
    f1 = f.read()  #接住
    f2 = ''.join(f1.split())  #把空白全去掉
    #print f2
    words=jieba.cut(f2)  #斷詞
    n = ' '.join(words)  #把斷詞結果用空白隔開
    n1 = n.encode('utf-8')  #編碼
    f.close()

    #統計字頻
    dic={}
    for w in n1.split(' '):
        if w not in dic:
            dic[w] = 1
        else:
            dic[w] = dic[w] + 1
    import operator
    words_freq = sorted(dic.iteritems(),key=operator.itemgetter(1),reverse=True)
    print attraction,"\n總詞量：",len(words_freq)

    #列出原始字頻排名
    print "==========字頻統計=========="
    print "排名","\t","詞彙","\t","字頻"
    print "- - - - - - - - - - - - - - "
    num2 = 0
    for ele in words_freq[0:rankNum]:
        num2 += 1
        print num2,"\t",ele[0],"\t",ele[1]
        if num2 % 10 == 0:
            print "- - - - - - - - - - - - - - "
            
# 比對多個景點斷詞字頻的方法
def mutifrestat(attrlist,RankNum):
    for i in range(0,len(attrlist)):
        jiebafrestat(attrlist[i],RankNum)
        print '\n'

In [31]:
lis = ['mountain.txt','lungshanTemple.txt','maokongGondola.txt']
mutifrestat(lis,10)

mountain.txt 
總詞量： 4487
==========字頻統計==========
排名 	詞彙 	字頻
- - - - - - - - - - - - - - 
1 	的 	2275
2 	。 	1564
3 	, 	850
4 	， 	527
5 	和 	501
6 	在 	416
7 	一個 	414
8 	我 	362
9 	是 	349
10 	你 	303
- - - - - - - - - - - - - - 


lungshanTemple.txt 
總詞量： 4970
==========字頻統計==========
排名 	詞彙 	字頻
- - - - - - - - - - - - - - 
1 	的 	2273
2 	。 	1546
3 	， 	831
4 	, 	745
5 	和 	495
6 	寺廟 	459
7 	在 	402
8 	是 	396
9 	一個 	364
10 	我 	322
- - - - - - - - - - - - - - 


maokongGondola.txt 
總詞量： 6484
==========字頻統計==========
排名 	詞彙 	字頻
- - - - - - - - - - - - - - 
1 	的 	4339
2 	。 	2915
3 	, 	1488
4 	， 	959
5 	和 	954
6 	一個 	736
7 	我們 	694
8 	在 	693
9 	是 	591
10 	纜車 	535
- - - - - - - - - - - - - - 




# 方法四 計算字頻 - 去除停用詞後的字頻統計

# 去停用詞的方法
# 法一 使用list轉set

def stopWordsList(stopwordfile):

    li = []
    fid =  open(stopwordfile,'r')
    for line in fid:
        li.append(line.strip())
    fid.close()
    setli = set(li)
    return setli

# 法一 使用set
def stopWordsList(stopwordfile):

    setli = set()
    fid =  open(stopwordfile,'r')
    for line in fid:
        setli.add(line.strip())
    fid.close()
    return setli
    
# 目前測試'法一'速度上略優於'法二'

In [10]:
def stopWordsList(stopwordfile):
    li = []
    fid =  open(stopwordfile,'r')
    for line in fid:
        li.append(line.strip())
    fid.close()
    setli = set(li)
    return setli

def jiebafrestopwords(attraction,stopwordfile,rankNum):
#轉換象山資料,並切詞,並使用JIEBA內建的tfidf統計排名
    f = open(attraction,'r')  #讀取檔案
    f1 = f.read()  #接住
    f2 = ''.join(f1.split())  #把空白全去掉
    #print f2
    words=jieba.cut(f2)  #斷詞
    n = ' '.join(words)  #把斷詞結果用空白隔開
    n1 = n.encode('utf-8')  #編碼
    f.close()

    #統計字頻
    dic={}
    comparelist = stopWordsList(stopwordfile)
    for w in n1.split(' '):
        if w not in comparelist:
            if w not in dic:
                dic[w] = 1
            else:
                dic[w] = dic[w] + 1
    import operator
    words_freq = sorted(dic.iteritems(),key=operator.itemgetter(1),reverse=True)
    print attraction,"\n總詞量(去除停用詞後)：",len(words_freq)

    #列出原始字頻排名
    print "==========字頻統計=========="
    print "排名","\t","詞彙","\t","字頻"
    print "- - - - - - - - - - - - - - "
    num2 = 0
    for ele in words_freq[0:rankNum]:
        num2 += 1
        print num2,"\t",ele[0],"\t",ele[1]
        if num2 % 10 == 0:
            print "- - - - - - - - - - - - - - "
            
# 比對多個景點斷詞字頻(去停用詞後)的方法
def mutifrestopwords(attrlist,stopWordsFile,RankNum):
    for i in range(0,len(attrlist)):
        jiebafrestopwords(attrlist[i],stopWordsFile,RankNum)
        print '\n'

In [24]:
import time
a = time.time()
lis = ['mountain.txt','lungshanTemple.txt','maokongGondola.txt']
mutifrestopwords(lis,'stopwords_1227.txt',10)
b = time.time()
print b-a

mountain.txt 
總詞量(去除停用詞後)： 3837
==========字頻統計==========
排名 	詞彙 	字頻
- - - - - - - - - - - - - - 
1 	101 	242
2 	象山 	156
3 	地方 	135
4 	城市 	123
5 	爬 	114
6 	值得 	97
7 	走 	88
8 	夜景 	87
9 	大象 	86
10 	檢視 	83
- - - - - - - - - - - - - - 


lungshanTemple.txt 
總詞量(去除停用詞後)： 4313
==========字頻統計==========
排名 	詞彙 	字頻
- - - - - - - - - - - - - - 
1 	寺廟 	459
2 	龍山寺 	213
3 	地方 	141
4 	臺灣 	105
5 	夜市 	94
6 	值得 	91
7 	捷運 	58
8 	遊客 	57
9 	文化 	56
10 	祈禱 	53
- - - - - - - - - - - - - - 


maokongGondola.txt 
總詞量(去除停用詞後)： 5723
==========字頻統計==========
排名 	詞彙 	字頻
- - - - - - - - - - - - - - 
1 	纜車 	535
2 	貓空 	344
3 	動物園 	298
4 	貢多拉 	277
5 	水晶 	205
6 	坐 	156
7 	站 	142
8 	風景 	138
9 	等待 	130
10 	玻璃 	124
- - - - - - - - - - - - - - 


0.680999994278


# 綜合比對
使用方法：

    1.新增一陣列：內含要分析景點的檔案名稱(TXT)
    2.決定要使用的停用詞字典(TXT)
    3.決定要顯示的排名數

使用範例：

    陣列 = [景點A檔,景點B檔,景點C檔]
    
    1.單一TFIDF
    mutijiebastat(陣列,顯示排名數)
    
    2.多重TFIDF
    CompareTFIDF(陣列,停用詞字典,顯示排名數)
    
    3.原始字頻統計
    mutifrestat(陣列,顯示排名數)
    
    4.去停用詞後字頻統計
    mutifrestopwords(陣列,停用詞字典,顯示排名數)
    
實作：

    list = ['mountain.txt','lungshanTemple.txt','maokongGondola.txt']
    rank = 10
    stpDic = 'stopwords_1222.txt'
    
    mutijiebastat(lis,rank)
    
    CompareTFIDF(lis,stpDic,rank)
    
    mutifrestat(lis,rank)
    
    mutifrestopwords(lis,stpDic,rank)

In [11]:
#實作：
import time

x = time.time()
lis = ['mountain.txt','lungshanTemple.txt','maokongGondola.txt']
rank = 10
stpDic = 'stopwords_1227.txt'

#法一
mutijiebastat(lis,rank)

#法二
CompareTFIDF(lis,stpDic,rank)

#法三
mutifrestat(lis,rank)

#法四
mutifrestopwords(lis,stpDic,rank)

y = time.time()
print 'Cost ',y-x,' secs.'

mountain.txt 
總詞量： 4487
=========字頻統計(根據jieba TFIDF)=========
排名 	總排 	詞彙 	出現次數    權重 			比率
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
1 	13 	101 	242 	    0.266101336985 	5.39 %
2 	23 	象山 	156 	    0.147262902206 	3.47 %
3 	47 	夜景 	87 	    0.0912661610321 	1.93 %
4 	48 	大象 	86 	    0.0796174348974 	1.91 %
5 	50 	檢視 	83 	    0.072573091905 	1.84 %
6 	54 	美景 	74 	    0.0695205415996 	1.64 %
7 	56 	日落 	73 	    0.0651036522607 	1.62 %
8 	60 	風景 	66 	    0.0630026876112 	1.47 %
9 	68 	很棒 	60 	    0.0615771688891 	1.33 %
10 	76 	臺北市 	56 	    0.061413294314 	1.24 %


lungshanTemple.txt 
總詞量： 4970
=========字頻統計(根據jieba TFIDF)=========
排名 	總排 	詞彙 	出現次數    權重 			比率
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
1 	6 	寺廟 	459 	    0.502494348336 	9.23 %
2 	13 	龍山寺 	213 	    0.233183651842 	4.28 %
3 	22 	地方 	141 	    0.114949687528 	2.83 %
4 	30 	臺灣 	105 	    0.086515551715 	2.11 %
5 	34 	夜市 	94 	    0.0634960178725 	1.89 %
6 	35 	值得 	91 	    0.0624012589437 	